In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install numpy==1.24.3 pandas scikit-learn scipy matplotlib pillow nibabel easydict
!pip install absl-py antlr4-python3-runtime anyio openai-clip einops fire ftfy huggingface-hub kornia lpips opencv-python omegaconf transformers==4.19.2 tensorboardx

Looking in indexes: https://download.pytorch.org/whl/cu118


In [2]:
!mkdir /content/extracted_features

In [3]:
!git clone https://github.com/Lojikick/fMRI_Yassification.git

Cloning into 'fMRI_Yassification'...
remote: Enumerating objects: 389, done.
remote: Counting objects: 100% (389/389), done.
remote: Compressing objects: 100% (310/310), done.
remote: Total 389 (delta 87), reused 360 (delta 61), pack-reused 0 (from 0)
Receiving objects: 100% (389/389), 6.40 MiB | 21.00 MiB/s, done.
Resolving deltas: 100% (87/87), done.


In [33]:
!git pull

remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 5 (delta 4), reused 5 (delta 4), pack-reused 0 (from 0)
Unpacking objects: 100% (5/5), 1.19 KiB | 1.19 MiB/s, done.
From https://github.com/Lojikick/fMRI_Yassification
   da62856..b5d7ccf  main       -> origin/main
Updating da62856..b5d7ccf
Fast-forward
 paper_code/scripts/versatilediffusion_reconstruct_images.py | 187 ++++++++++++++++++--------------
 1 file changed, 106 insertions(+), 81 deletions(-)


In [4]:
%cd fMRI_Yassification/paper_code/

/content/fMRI_Yassification/paper_code


In [5]:
import easydict

In [22]:
import sys
sys.path.append('versatile_diffusion')

In [ ]:
from lib.cfg_helper import model_cfg_bank

In [ ]:
import torch

# Check CUDA availability
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")

# Your device-agnostic setup
def get_device():
    if torch.cuda.is_available():
        return torch.device("cuda")
    elif torch.backends.mps.is_available():
        return torch.device("mps")
    else:
        return torch.device("cpu")

device = get_device()
print(f"Using device: {device}")

CUDA available: True
CUDA device: Tesla T4
CUDA version: 12.1
Using device: cuda


In [ ]:
import gdown

# Download cliptext_features
!gdown '1XrnndoGIx26zCIRQuGfKhvPD39G4Ze9P' -O nsd_cliptext_train.zip
# Unzip file to /content
!unzip nsd_cliptext_train.zip -d /content

Downloading...
From (original): https://drive.google.com/uc?id=1XrnndoGIx26zCIRQuGfKhvPD39G4Ze9P
From (redirected): https://drive.google.com/uc?id=1XrnndoGIx26zCIRQuGfKhvPD39G4Ze9P&confirm=t&uuid=e13d1817-f464-40cd-b1a7-8b4b85aa952a
To: /content/fMRI_Yassification/paper_code/nsd_cliptext_train.zip
100% 2.27G/2.27G [00:10<00:00, 217MB/s] 
Archive:  nsd_cliptext_train.zip
  inflating: /content/nsd_cliptext_train.npy  


In [19]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [9]:
import zipfile
import os

In [6]:
import gdown

# Download first file (subj01.zip)
!gdown '1JOQ2JNMWS_jPrseJWQn0QF2mtXVktZAN' -O nsd_cliptext_predtest_nsdgeneral.zip

# Download second file (versatile_diff.zip)
!gdown '1Dw4QNK7yJVEitIZq-njNal1GsFMrJ7NG' -O nsd_clipvision_predtest_nsdgeneral.zip

# Unzip both files to /content
!unzip nsd_cliptext_predtest_nsdgeneral.zip -d /content
!unzip nsd_clipvision_predtest_nsdgeneral.zip -d /content

Downloading...
From (original): https://drive.google.com/uc?id=1JOQ2JNMWS_jPrseJWQn0QF2mtXVktZAN
From (redirected): https://drive.google.com/uc?id=1JOQ2JNMWS_jPrseJWQn0QF2mtXVktZAN&confirm=t&uuid=ebfa64ef-9926-418b-8dc5-992899124c95
To: /content/fMRI_Yassification/paper_code/nsd_cliptext_predtest_nsdgeneral.zip
100% 445M/445M [00:07<00:00, 60.9MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1Dw4QNK7yJVEitIZq-njNal1GsFMrJ7NG
From (redirected): https://drive.google.com/uc?id=1Dw4QNK7yJVEitIZq-njNal1GsFMrJ7NG&confirm=t&uuid=53e1234a-10d8-42b0-9f0a-c3722cf658c8
To: /content/fMRI_Yassification/paper_code/nsd_clipvision_predtest_nsdgeneral.zip
100% 1.49G/1.49G [00:16<00:00, 88.9MB/s]
Archive:  nsd_cliptext_predtest_nsdgeneral.zip
  inflating: /content/nsd_cliptext_predtest_nsdgeneral.npy  
Archive:  nsd_clipvision_predtest_nsdgeneral.zip
  inflating: /content/nsd_clipvision_predtest_nsdgeneral.npy  


In [7]:
def unzip_drive_files(zip_path, extract_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
        print(f'Extracted: {zip_path} to {extract_path}')

In [12]:
specific_zip = '/content/drive/MyDrive/brain_diffuser_models/versatile_diff.zip'
extract_to = '/content/'
os.makedirs(extract_to, exist_ok=True)
unzip_drive_files(specific_zip, extract_to)

Extracted: /content/drive/MyDrive/brain_diffuser_models/versatile_diff.zip to /content/


In [20]:
specific_zip = '/content/drive/MyDrive/brain_diffuser_models/test_photos.zip'
extract_to = '/content/'
os.makedirs(extract_to, exist_ok=True)
unzip_drive_files(specific_zip, extract_to)

Extracted: /content/drive/MyDrive/brain_diffuser_models/test_photos.zip to /content/


In [23]:
sys.path.append('scripts')

In [25]:
!mv /content/nsd_* /content/fMRI_Yassification/paper_code/data/predicted_features/subj01/

In [26]:
!mv /content/versatile_diff/* /content/fMRI_Yassification/paper_code/versatile_diffusion/pretrained/

In [29]:
!mv /content/*.png /content/fMRI_Yassification/paper_code/results/vdvae/subj01

In [ ]:
!python scripts/versatilediffusion_reconstruct_images.py -sub 1

Available GPUs: 1
Current GPU: 0
GPU Name: Tesla T4

#######################
# Running in eps mode #
#######################

Load openai_unet_2d with total 859520964 parameters,100526.846 parameter sum.
Load openai_unet_0dmd with total 1708848448 parameters,249653.968 parameter sum.
Load openai_unet_vd with total 2566318852 parameters,350178.249 parameter sum.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels
/content/fMRI_Yassification/paper_code/versatile_diffusion/lib/model_zoo/common/get_model.py:97: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, th

In [ ]:
!python scripts/clipvision_extract_features.py -sub 1


#######################
# Running in eps mode #
#######################

Load openai_unet_2d with total 859520964 parameters,99943.638 parameter sum.
Load openai_unet_0dmd with total 1708848448 parameters,250386.806 parameter sum.
Load openai_unet_vd with total 2566318852 parameters,350336.815 parameter sum.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels
/content/fMRI_Yassification/paper_code/versatile_diffusion/lib/model_zoo/common/get_model.py:97: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to

In [ ]:
!ls

fMRI_Yassification  __MACOSX  sample_data  subj01


In [ ]:
%cd fMRI_Yassification/paper_code

/content/fMRI_Yassification/paper_code


In [ ]:
# Now you should be able to import
from lib.cfg_helper import model_cfg_bank

In [ ]:
#CLIP Vision extraction Processing:
import sys
import numpy as np
import os
import PIL
from PIL import Image

In [ ]:
train_clip = np.zeros((5, 10))
sub = 1  # Example subject number

In [ ]:
save_path = f'/content/extracted_features/subj01'
os.makedirs(save_path, exist_ok=True)


In [ ]:
np.save(f'{save_path}/nsd_cliptext_train.npy', train_clip)

In [ ]:
loaded_data = np.load(f'{save_path}/nsd_cliptext_train.npy')

In [ ]:
loaded_data = np.load('/content/subj01/nsd_test_cap_sub1.npy')

In [ ]:
loaded_data

array([982,   5])

In [ ]:
%cd /content/drive/MyDrive/brain_diffuser_data/lib

/content/drive/MyDrive/brain_diffuser_data/lib


In [ ]:
!ls

cfg_helper.py  data_factory  experiments  log_service.py  __pycache__  utils.py
cfg_holder.py  evaluator     __init__.py  model_zoo	  sync.py


In [ ]:
%cd /content/drive/MyDrive/brain_diffuser_data/

/content/drive/MyDrive/brain_diffuser_data


In [ ]:
#GOAL: Get the text extraction to work

In [ ]:
#CLIP Vision extraction Processing:
import sys
import numpy as np
import os
import PIL
from PIL import Image

In [ ]:
from lib.cfg_helper import model_cfg_bank

ModuleNotFoundError: No module named 'lib.cfg_helper'

In [ ]:
train_clip = np.zeros((5, 10))
sub = 1  # Example subject number


NameError: name 'np' is not defined

In [ ]:
import torch
from lib.cfg_helper import model_cfg_bank
from lib.model_zoo import get_model
from lib.experiments.sd_default import color_adjust, auto_merge_imlist
from torch.utils.data import DataLoader, Dataset

from lib.model_zoo.vd import VD
from lib.cfg_holder import cfg_unique_holder as cfguh
from lib.cfg_helper import get_command_line_args, cfg_initiates, load_cfg_yaml
import torchvision.transforms as T
pip

import argparse

ModuleNotFoundError: No module named 'lib.cfg_helper'

In [ ]:
train_stim = np.load('/content/subj01/nsd_train_stim_sub1.npy')
test_stim = np.load('/content/subj01/nsd_test_stim_sub1.npy')

In [ ]:
print(train_captions)

[8859    5]


In [ ]:
# Data Processing for .npy file here!

In [ ]:
import numpy as np
